First, move within a simulation directory, which contains the directory 'output/' that contains snapshot files, and a file 'snapshot_times.txt' that lists all snapshot scale-factors/redshifts/times in the simulation and their corresponding file index number.

Ensure that gizmo_analysis and utilities directories are in your python path, then...

In [ ]:
#import gizmo_analysis as gizmo
import gizmo  # I happen to save my 'gizmo_analysis' directory as 'gizmo', but do as you prefer.

import utilities as ut

In [ ]:
# you can access the files as named or use the aliases in __init__.py to keep it shorter 
# for example, these are the same:

gizmo.gizmo_io
gizmo.io

# read particle data

In [ ]:
# read star and dark matter particles at z = 0

part = gizmo.io.Read.read_snapshots(['star', 'dark'], 'redshift', 0)

In [ ]:
# alternately, read all particle species at z = 0

part = gizmo.io.Read.read_snapshots('all', 'redshift', 0)

In [ ]:
# each particle species is stored as its own dictionary
# 'star' = stars, 'gas' = gas, 'dark' = dark matter, 'dark.2', 'dark.3', etc = low-resolution dark matter

part.keys()

In [ ]:
# properties of star particles are stored as dictionary

part['star'].keys()

In [ ]:
# properties of dark matter particles are stored as dictionary

part['dark'].keys()

# particle properties

In [ ]:
# 3-D position of star particles (particle.number x dimension.number array) [Mpc comoving]

part['star']['position']

In [ ]:
# 3-D velocity of star particles (particle_number x dimension_number array) [km/s physical]

part['star']['velocity']

In [ ]:
# mass of star particles [M_sun]

part['star']['mass']

In [ ]:
# formation scale-factor of star particles

part['star']['form.scalefactor']

In [ ]:
# use .prop() to compute derived quantities,
# such as the age of the Universe [Gyr] when a star particle formed
# see gizmo.io.ParticleDictionaryClass for all options for derived quantities

part['star'].prop('form.time')

In [ ]:
# similarly, get the age of a star particle (the lookback time to when it formed) [Gyr]

part['star'].prop('age')

# metallicities

In [ ]:
# 'metallicities' are stored in the catalog as *mass fractions*
# one value for each element, in a particle_number x element_number array
# the first value is the mass fraction of all metals (everything not H, He)
# then He, C, N, O, etc

part['star']['massfraction']

In [ ]:
# get individual elements by their index

# total metal mass fraction (everything not H, He) is index 0
print(part['star']['massfraction'][:, 0])

# iron is index 10
print(part['star']['massfraction'][:, 10])

In [ ]:
# alternately use .prop() to compute derived quantities,
# including calling elements by their name
# see gizmo.io.ParticleDictionaryClass for all options for derived quantities

print(part['star'].prop('massfraction.metals'))
print(part['star'].prop('massfraction.carbon'))
print(part['star'].prop('massfraction.iron'))

In [ ]:
# also use .prop() to compute metallicity [Z/H]
# for example, iron metallicity [Fe/H] :=
#   log10((mass_iron / mass_hydrogen)_particle / (mass_iron / mass_hydrogen)_sun)

print(part['star'].prop('metallicity.total'))
print(part['star'].prop('metallicity.fe'))

In [ ]:
# also use .prop() to compute simple arithmetic combinations, such as [Mg/Fe]

part['star'].prop('metallicity.mg - metallicity.fe')

In [ ]:
# refer to ut.basic.constants for assumed solar values (and other constants)

ut.basic.constants.sun_composition

# additional information stored in sub-dictionaries

In [ ]:
# dictionary of useful information about the simulation

part.info

In [ ]:
# dictionary of information about this snapshot's index, scale-factor, redshift, time, lookback-time

part.snapshot

In [ ]:
# dictionary of arrays about *all* snapshots that were saved for the simulation

print(part.Snapshot.keys())
print(part.Snapshot['redshift'][:10])

In [ ]:
# position [kpc comoving] and velocity [km/s physical] of the center of the host galaxy
# this was computed during read in, using ut.particle.get_center_position() and ut.particle.get_center_velocity()
# functions in gizmo.analysis use these values in computing profiles

print(part.center_position)
print(part.center_velocity)

See gizmo.analysis for examples of high-level analysis, including plotting these data.

See ut.particle for mid-level analysis functions that may be useful.

See other modules within utilities for low-level functions that may be useful.

# principal (major and minor axes) of stellar disk

In [ ]:
# can compute and assign principal axes (moment of inertia tensor) of stars during read in as below

part = gizmo.io.Read.read_snapshots(['star', 'dark'], 'redshift', 0, assign_principal_axes=True)

In [ ]:
# moment of inertia tensor is stored similar to center_position

print(part.principal_axes_vectors)

In [ ]:
# now can compute different types of distances of star particles from center of main galaxy
# this computes 3-D distance from main galaxy center along simulation's default x,y,z axes [kpc physical]

ut.particle.get_distances_wrt_center(part, 'star', 'vector')

In [ ]:
# compute 3-D distances along the principal (major, intermediate, minor) axes

ut.particle.get_distances_wrt_center(part, 'star', 'rotated')

In [ ]:
# compute 2-D distances along the major axes (major + intermediate) and minor axis
# first value is distance (positive definite) [kpc physical] along the major axes
# second value is (signed) vertical height [kpc physical] wrt the disk

ut.particle.get_distances_wrt_center(part, 'star', 'rotated.2d')

# particle tracking

For some simulations, I generated pre-compiled HDF5 files to help with particle tracking. These are stored in the directory 'track/' (if present). The code that generates and reads these files is in gizmo_track.py.

star\_indices\_\*.hdf5 files store, for each star particles at z = 0, a pointer to where it was in the catalog at each previous snapshot (replace * with snapshot index). This makes it easy to quickly get the properties of a given star particle at any previous snapshot. These pointers are stored in an HDF5 file, one for each previous snapshot.

In [ ]:
# first, read catalog of star particles at z = 0

part_at_z0 = gizmo.io.Read.read_snapshots(['star'], 'redshift', 0)

In [ ]:
# say that you want to find out what they were doing at z = 1
# then, read in catalog of star particles at z = 1 (snapshot 277)

part_at_z1 = gizmo.io.Read.read_snapshots(['star'], 'redshift', 1)

In [ ]:
# use the function within gizmo_track.py to read star index pointers associated with the catalog z = 1

gizmo.track.IndexPointer.io_index_pointer(part_at_z1)

In [ ]:
# pointers are stored via numpy array appended to particle dictionary at the relevant snapshot
# a negative value means that the star formed after this snapshot (so it does not exist at this snapshot)

part_at_z1.index_pointers

In [ ]:
# so, say that you have a list of the indices of star particles of interest at z = 0

indices_at_z0 = np.array([3, 5, 8, 13])

In [ ]:
# their positions at z = 0

part_at_z1['star']['position'][indices_at_z0]    

In [ ]:
# get their indices in the catalog at z = 1

indices_at_z1 = part_at_z1.index_pointers[indices_at_z0]
print(indices_at_z1)

In [ ]:
# now you easily can get any property of interest at z = 1, for example, positions

part_at_z1['star']['position'][indices_at_z1]    

Another useful file: star\_form\_host\_distance\_600.hdf5 stores, for each star particle at z = 0, its distance at the first snapshot after it formed (formation distance) wrt to the main host galaxy, in kpc physical. This file includes both total (scalar) distance and full 3-D vector distance.

In [ ]:
# use the function within gizmo_track.py to read this file and assign values directly to the catalog at z = 0

gizmo.track.HostDistance.io_form_host_distance(part_at_z0)

In [ ]:
# total (scalar) distance at formation, wrt host galaxy [kpc physical]

part_at_z0['star']['form.host.distance']

In [ ]:
# 3-D distance vector at formation, wrt host galaxy [kpc physical]
# these distances are aligned with the principal axes of the host galaxy,
# principal axes defined according to all star particles within the host galaxy, independently at each snapshot
# distance along dimension 0 is aligned with the major axis
# distance along dimension 1 is algined with the intermediate axis
# distance along dimension 2 is aligned with the minor (Z) axis

part_at_z0['star']['form.host.distance.3d']

In [ ]:
# for a disk galaxy, you may want to combine the major and intermediate axes into an absolute 'radius'
# this function is useful: it returns a 2-D array scalar (absolute) radius and (signed) vertical height within the disk

distances_2d = ut.coordinate.get_distances_major_minor(part_at_z0['star']['form.host.distance.3d'])
print(distances_2d[:, 0]) # absolute 2-D radius R
print(distances_2d[:, 1]) # signed vertical height Z

In [ ]:
# these value look more reasonable is restrict to star particles that formed within the host galaxy

# select particles formed at d = 0 - 8 kpc physical
part_indices = ut.array.get_indices(part_at_z0['star']['form.host.distance'], [0, 8])

# get current position in the disk
distances_2d = ut.coordinate.get_distances_major_minor(part_at_z0['star']['form.host.distance.3d'][part_indices])
print(distances_2d[:, 0]) # absolute 2-D radius R
print(distances_2d[:, 1]) # signed vertical height Z